### Model Validation: Merton Model default probabilities correlate with observed CDS spreads.

### Probably now should be looking to see how merton model probs compare to bloomberg probs

# DELETE LATER
Expected Result: positive correlation


In [25]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import roc_auc_score

In [ ]:
df= pd.read_csv('../model/merton_model_output.csv')
df_valid = df[df['merton_pd'].notna()].copy()
df_valid['log_E'] = np.log(df_valid['market_cap'])

In [ ]:
bb_pd = pd.read_csv('./DEFAULT.csv')

C:\Users\tkpma\AppData\Local\Temp\ipykernel_15916\3660149276.py:1: DtypeWarning: Columns (4,6,20,22,24,26,28,34,36,38,40,42,44,46,48,50,54,58,60,62,64,66,68,72,92,94,104,106,112,114,120,126,128,132,134,136,140,146,150,152,156,158,166,168,178,182,186) have mixed types. Specify dtype option on import or set low_memory=False.
  bb_pd = pd.read_csv('../hypothesis-4/DEFAULT.csv')


Organize Bloomberg Data into a usable df

In [7]:
empirical_probs = pd.DataFrame()
empirical_probs.index = bb_pd['Date']
empirical_probs.index.values[0] = '1/3/2005'
empirical_probs.index = pd.to_datetime(empirical_probs.index, format='%m/%d/%Y')
filtered_cols = [col for col in bb_pd.columns if 'Date' not in col]
empirical_probs[bb_pd[filtered_cols].columns] = bb_pd[filtered_cols].values
empirical_probs = empirical_probs.dropna(axis=1)

Organize Merton Model

In [9]:
merton = pd.DataFrame()
merton[empirical_probs.columns] = empirical_probs.columns
for col in merton.columns:
    matches = df_valid.loc[df_valid['tic'] == col, 'merton_pd'].reset_index(drop=True)
    merton[col] = matches

merton.index = empirical_probs.index

In [10]:
combined = pd.concat([empirical_probs, merton], axis=1, keys=['Empirical','Model']).swaplevel(axis=1).sort_index(axis=1)


Compute Correlation Coefficient between Model and Empirical Data

In [21]:
x = empirical_probs.values.flatten()
y = merton.values.flatten()
mask = ~ np.isnan(x) & ~np.isnan(y)

spearman_corr, p_value = spearmanr(x[mask],y[mask])

In [23]:
spearman_corr

np.float64(0.4989079543287979)